## Preparing the dataset
Converting the CSV File into text label files for YOLO

In [ ]:
import os
import pandas as pd

csv_file_path = './train/labels.csv'
images_path = './train/train'

data = pd.read_csv(csv_file_path)

def convert(xmin, ymin, xmax, ymax):
    x_center, y_center = (xmin + xmax) / 2, (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return [0, x_center, y_center, width, height]

images = os.listdir('train/images')
labels = os.listdir('train/labels')

for i, r in data.iterrows():
    filename = r[0]
    lab = convert(r[1], r[2], r[3], r[4])
    label = (' '.join(map(str, lab)) + ' ')

    with open(f"train/train_labels/{filename}.txt", 'w') as file:
        file.write(label)

## Making the Train, Val and Test Folders 

In [ ]:
import os

split_index = int(len(images) * 0.8)
# split_index

train_images = images[:split_index]
train_labels = labels[:split_index]

val_images = images[split_index:]
val_labels = labels[split_index:]

# Moving the images to val
# for filename in val_images:
#     os.rename(f"./train/images/{filename}", f"./val/images/{filename}")
    
# Moving the labels to val
# for filename in val_labels:
#     os.rename(f"./train/labels/{filename}", f"./val/labels/{filename}")

## Normalising all values in the files between 0 and 1

In [ ]:
import os
import numpy as np

image_width = 640
image_height = 480

def normalize_values(values):
     # YOLO format: class_id, x_center, y_center, width, height
    class_id = values[0]
    x_center = values[1] / image_width
    y_center = values[2] / image_height
    width = values[3] / image_width
    height = values[4] / image_height
    
    return [class_id, x_center, y_center, width, height]

trainLabelsPath = os.path.join('./train/labels')
trainLabels = os.listdir(trainLabelsPath)
valLabelsPath = os.path.join('./val/labels')
valLabels = os.listdir(valLabelsPath)

# Writing the normalised Training Labels
# for filename in trainLabels:
    
#     filename = os.path.join(trainLabelsPath, filename)
    
#     with open(filename, 'r') as file:
#         fileData = file.read().split()
#         values = np.array(fileData, dtype=float)
#         values = normalize_values(values)
#     file.close()
    
#     normalized_values = list(map(str, values))
    
#     with open(filename, 'w') as f:
#         f.write(' '.join(normalized_values) + '\n')
#     f.close()


# Writing the normalised Validation Labels
for filename in valLabels:
    
    filename = os.path.join(valLabelsPath, filename)
    
    with open(filename, 'r') as file:
        fileData = file.read().split()
        values = np.array(fileData, dtype=float)
        values = normalize_values(values)
    file.close()
    
    normalized_values = list(map(str, values))
    
    with open(filename, 'w') as f:
        f.write(' '.join(normalized_values) + '\n')
    f.close()

## Checking the bounding boxes

In [ ]:
import os
from PIL import Image, ImageDraw

# Define paths
image_dir = './train_copy/images'
label_dir = './train_copy/labels'

# List of image and label files
image_files = os.listdir(image_dir)
label_files = os.listdir(label_dir)

# Ensure only to process the first 10 images
image_files = image_files[:10]

# Iterate over the first 10 images
for image_file in image_files:
    # Load image
    img_path = os.path.join(image_dir, image_file)
    img = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    # Get corresponding label file
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_path = os.path.join(label_dir, label_file)

    # Read labels
    with open(label_path, 'r') as f:
        labels = f.readlines()

    # Draw bounding boxes
    for label in labels:
        label_data = label.split()
        class_id = int(label_data[0])
        x_center, y_center, width, height = map(float, label_data[1:])

        # Calculate coordinates
        x_min = x_center - (width / 2)
        y_min = y_center - (height / 2)
        x_max = x_center + (width / 2)
        y_max = y_center + (height / 2)

        # Draw rectangle
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)

    # Display image
    img.show()

## Training it on YOLO v8
Now we will load the YOLO v8 model and train it on the custom dataset to obtain new weights, to get bounding boxes, identifying the foot ulcers

In [1]:
from ultralytics import YOLO

model = YOLO('models/yolov8n.pt')

res = model.train(data='data.yaml', epochs=250, batch=25, imgsz=640)

Ultralytics YOLOv8.2.78  Python-3.11.5 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=models/yolov8n.pt, data=data.yaml, epochs=250, time=None, patience=100, batch=25, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aayudhpanchal (aayudhpanchal-sardar-vallabhbhai-national-institute-of-t). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed 


train: Scanning C:\Users\aayud\OneDrive\Desktop\VS Code\Machine Learning\Internship\Diabetic Foot Ulcer Detection\train\labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]
val: Scanning C:\Users\aayud\OneDrive\Desktop\VS Code\Machine Learning\Internship\Diabetic Foot Ulcer Detection\val\labels.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005859375000000001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        400        400      0.995      0.995      0.994      0.918



250 epochs completed in 1.458 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.3MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.78  Python-3.11.5 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.06it/s]


                   all        400        400      0.992          1      0.994      0.916
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train


lr/pg0,▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▄▅▅▆▆▆▆▆▇▇▇▇███████████████████████████
metrics/mAP50-95(B),▁▂▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████████
metrics/precision(B),▁▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████████████████████
metrics/recall(B),▁▃▄▄▅▅▆▅▆▆▆▇▇▇▇▇▇▇▇▇████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
